<a href="https://colab.research.google.com/github/ev1025/DA_Study/blob/main/%ED%81%AC%EB%A1%A4%EB%A7%81_%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install webdriver_manager

In [ ]:
pip install selenium

In [ ]:
pip install BeautifulSoup4

# 카카오맵 크롤링

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import requests
import time
import math
import re
import pandas as pd

# 크롬 옵션
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('--disable-gpu--')
options.add_argument('lang=ko_KR')

# 크롬 드라이버 설치
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 카카오맵 방문
driver.get("https://map.kakao.com/")

# 페이지 로드시간 암묵적 대기
driver.implicitly_wait(10)

# csv 입력받을 DataFrame
dog_map_df = pd.DataFrame(columns=('name','star','adr','category'))

# 검색 할 목록
search_list = ['서울 애견 미용실','서울 애견호텔', '서울 반려동물 미용실', '서울 동물약국',\
                 '서울 동물병원', '서울 강아지 수제간식','서울 애견 수제간식', '서울 애견동반', '서울 강아지유치원','서울 애견유치원', '서울 애견용품']

# 목록에서 하나씩 검색
for search_id in search_list:
    # 검색상자 찾기
    search_box = driver.find_element(By.CSS_SELECTOR, "#search\.keyword\.query")
    # 검색할 단어 입력
    search_box.send_keys(f"{search_id}")
    time.sleep(1)
    # 검색버튼 누르기
    search_box.send_keys(Keys.ENTER)
    time.sleep(1)

    # 지도설정 팝업 제거(최초 1회만 나타나서 예외구문 적용)
    try:
        driver.find_element(By.CSS_SELECTOR, "body > div.coach_layer.coach_layer_type1 > div > div > div > span").click()
    except:
        pass
    time.sleep(1)

    # 최초에 페이지번호 뜨지 않고 더보기 눌러줘야함(최초 1회만 나타나서 예외구문 적용)
    try:
        more_btn = driver.find_element(By.CSS_SELECTOR, "#info\.search\.place\.more")
        more_btn.click()
    except:
        pass
    time.sleep(1)

    # 검색결과의 총 개수 / 75(5페이지 당 요소 개수)
    all_page = "document.querySelector('.keywordSearch').innerHTML"
    page_raw = driver.execute_script("return " + all_page)
    page_html = BeautifulSoup(page_raw, 'html.parser')
    all_page_html = page_html.find('em', class_="cnt", id="info.search.place.cnt").text
    all_page_html = re.sub(r'[^0-9]', '', all_page_html)
    all_page_num = math.ceil(int(all_page_html.text) / 75)

    # dog_map_df에 입력받을 컬럼
    name = []
    star = []
    adr = []
    category = []         
    
    # 크롤링 시작
    # 페이지당 15개의 개체가 있다.
    # 이전, 페이지 1~5, 다음 버튼이 존재하며, 이전,다음버튼은 5개씩 넘어가게 되어있음
    
    # 다음버튼 누르는 횟수 개체 개수 = (15개 x 5page x 다음버튼 횟수)
    for p in range(all_page_num):
        # 1page ~ 5page 페이지이동 (6~10, 11~15 ...)
        # page버튼이 모자르면 종료(6~10구간에서 6,7,8까지만 있으면 8page에서 종료)
        for j in range(1,6):
            try:
                next_btn_1stp = driver.find_element(By.CSS_SELECTOR, f"#info\.search\.page\.no{j}")
                next_btn_1stp.click()
            except:
                print(f'{search_id}수집 끝')
                break
            time.sleep(1)

            # 추출할 범위를 CSS로 선택해서 HTML형식으로 변환
            js_script = "document.querySelector('.placelist').innerHTML"
            # 자바스크립트형식 읽어들이기
            raw = driver.execute_script("return " + js_script)
            html = BeautifulSoup(raw, 'html.parser')
            time.sleep(2)

            # 직접적인 데이터가 있는 CSS그룹 선택
            contents = html.find_all(class_='PlaceItem clickArea')
            time.sleep(2)

            # CSS그룹에서 필요한 값 추출
            for i in contents:
                cr_name = i.find(class_='link_name').text
                name.append(cr_name)

                try:
                    cr_star = float(i.select_one("em.num").text)
                    star.append(cr_star)
                except:
                    star.append(0)

                cr_adr = i.select_one('.addr').text.replace('\n', '').split(' ')
                if cr_adr[0] == '서울':
                    adr.append(cr_adr[1])

                cr_category = i.find(class_='subcategory clickable').text
                category.append(cr_category)
                

        next_btn = driver.find_element(By.XPATH,'//*[@id="info.search.page.next"]')
        next_btn.send_keys(Keys.ENTER)

   
    # 추출한 데이터 입력
    search_data = pd.DataFrame({'name':name,'star':star,'adr':adr,'category':category})
    dog_map_df = pd.concat([dog_map_df,search_data]) 
  
    # 검색어 지우기
    driver.implicitly_wait(3)
    search_box.clear()
    

# 중복제거
dog_map_df.drop_duplicates(['name'], keep='first', inplace=True)
``
# csv파일로 저장
dog_map_df.to_csv(path_or_buf=f'./dog_map_data.csv',encoding='utf-8-sig', index=False)

# 크롬 웹페이지를 닫음
driver.close()

### 네이버 API

In [ ]:
import requests
from urllib.parse import quote     # 한글 -> ASCII코드
import pandas as pd
import csv

# 네이버 API 아이디, 비번
client_id = ""
client_secret = ""

headers = {
   'X-Naver-Client-Id':  client_id,
   'X-Naver-Client-Secret': client_secret
}
# API주소
url = "https://openapi.naver.com/v1/search/local.json"

# 검색어 목록
search =['서울 동물병원']

# 검색어 마다 실행
for query in search:
    disp = '5'       # 출력개수(없어도됨)
    start = '1'      # 시작번호(없어도됨)

    params = {'query' : query,
            'display' : disp,
            'start' : start}

    resp = requests.get(url, headers=headers, params=params)

    # if resp.status_code ==200:
    #   print(resp.json().get('items'))


    # 요청결과 저장
    search_list = []

    for i in range(5):
        temp = []
        temp.append(resp.json().get('items')[i]['title'])
        temp.append(resp.json().get('items')[i]['category'])
        temp.append(resp.json().get('items')[i]['address'].split()[0])
        temp.append(resp.json().get('items')[i]['address'].split()[1])
        temp.append(resp.json().get('items')[i]['mapx'])
        temp.append(resp.json().get('items')[i]['mapy'])
        temp.append(resp.json().get('items')[i]['link'])
        search_list.append(temp)

    # csv 저장
    # 파일명 /'w'쓰기모드 / 한글인코딩 / 자동줄바꿈 하지않음
    f = open(f'{query}.csv', 'w', encoding='utf-8-sig', newline='') 
    csvwriter = csv.writer(f)

     # 컬럼명 입력
    csvwriter.writerow(['이름','업종','시','행정구','mapx','mapy','링크주소'])

    # 행입력(위에 search_list 값 입력)
    for j in search_list: 
        csvwriter.writerow(j)
        
    f.close()

    # 하나의 검색어 저장이 완료되면 출력
    print(f'{query} 완료')

서울 동물병원 완료


### 탑텐몰 제품, 가격, 할인률

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd

driver = webdriver.Chrome("C:/Users/eg287/chromedriver") # 나의 웹드라이버의 위치(설정-도움말-버전에서 확인 후 최신 드라이버 사용)
driver.get("https://www.topten10mall.com/kr/front/search/categorySearch.do?ctgNo=37341") # 웹사이트 방문

# 팝업 창 제거
# driver.find_element(By.CSS_SELECTOR, "button#intro_popup_close").click()
driver.implicitly_wait(10) # 페이지 로드시간에 10초 암묵적 대기

# 검색창에 검색어 입력하기
search_box = driver.find_element(By.CSS_SELECTOR, "#searchWord")
search_box.send_keys("맨투맨")

time.sleep(3)

# 검색버튼 누르기
search_box.send_keys(Keys.ENTER)
# raw = driver.page_source # 현재 랜더링된 페이지의 정보를 가져옴

# 추출한 데이터 저장할 DataFrame 생성
empty = pd.DataFrame(columns=['name','price','discount'])
num = 0 # 추출한 값을 넣을 인덱스번호

# 크롤링
for p in range(28):
    # 5초 delay
    time.sleep(2)
    
    # 자바로 HTML가져오기(추출할 범위)
    js_script = "document.querySelector('#divList').innerHTML"
    raw = driver.execute_script("return " + js_script)
    
    # requests로 HTML 가져오기
    # raw = requests.get("https://www.topten10mall.com/kr/front/search/totalSearch.do?searchTerm=%EC%85%94%EC%B8%A0")

    # 파서로 추출할 대상 선택
    html = BeautifulSoup(raw, 'html.parser')
    contents = html.select('.card-goods__body')

    # 추출대상별 구체적으로 추출할 클래스 값 추출
    for i in contents:
        name = i.select_one('.card-goods__text').text
        try:
            price = i.select_one('.card-goods__price').text
        except:
            price = 'NULL'
        try:
            discount = i.select_one('.card-goods__discount').text
        except:
            discount = '0%'

        # empty 데이터프레임에 추출한 값 넣기
        empty_dict = {'name':name, 'price':price, 'discount':discount}
        for key,value in empty_dict.items():
            empty.loc[num,key] = value
        num += 1 # 인덱스 번호 1씩 증가하도록
        # print(name, price, discount)
        
        
    # 다음 페이지로 이동(다음페이지 없을때까지)
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, '#searchGoods > nav > ul > li:nth-last-child(1)')
        next_btn.click()
    except:
        print("데이터 수집 완료")
        break


# empty.to_csv(path_or_buf='./크롤링.csv',index=False, encoding = "utf-8-sig")
len(empty)

# 네이버 강아지도 크롤링

서울지역 url 가져오기

In [ ]:
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://campaign.naver.com/save-the-stars/"

def get_page(page_url):
    page = requests.get(page_url) # page에 response와 html객체를 담는다
    soup = BeautifulSoup(page.content, 'html.parser') # html객체를 .content로 문자열화하고, html.parser로 파싱 (DOM객체)
    return soup, page 

soup, page = get_page(BASE_URL)
dogMapList = soup.select("#new li.items > a") 
# >는 직계자손관계, 단순 띄어쓰기는 자식 손주 증손주 등등 다 가능
# select는 element들을 가져와서 list화

def get_link(item): 
    return item.attrs["href"] # attrs(속성)을 가져와라

def is_Seoul(item):
    return '서울' in item.select(".sub_tit")[0].text

links = list(map(get_link, list(filter(is_Seoul, dogMapList))))

가게이름 가져오기

In [ ]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

# 웹드라이버
wd = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = 'https://naver.me/5WBTxFeW'

# 브라우저가 url을 방문, 페이지 로드 시간 대기
wd.get(url)
time.sleep(5)

# ======================= [START] html 코드를 가져오기 전 웹을 조작하는 코드 =======================

# iframe의 id를 이용해서 frame focus 전환 
wd.switch_to.frame("myPlaceBookmarkListIframe")

# 스크롤 코드 출처 : https://hello-bryan.tistory.com/194
# 스크롤 -> 필요한 만큼 반복문으로 구성

# 스크롤 높이 가져옴
last_height = wd.execute_script("return document.body.scrollHeight")
while True:
    # 아래코드는 자바스크립트의 scroll 함수를 사용하기 위해 자바스크립트 코드를 web driver에게 전달
    # 자바스크립트 코드의 기능 -> 끝까지 스크롤 다운
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    
    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = wd.execute_script("return document.body.scrollHeight")
    
    # 이전 height와 새로운 height와 같으면 탈출
    # 스크롤 할때마다 전체 height가 커지기 때문에 이전 height와 같다는 뜻은 더 이상 스크롤이 없다는 뜻
    if new_height == last_height:
        break
    last_height = new_height

# ======================= [END] html 코드를 가져오기 전 웹을 조작하는 코드 =======================

# web driver -> js 코드 전달
# web 드라이버한테 내가 필요한 부분의 html 코드를 문자열로 받는게 목표!
# 그걸 위해서 내가 필요한 부분을 선택하는 자바스크립트 코드를 web driver에게 전달하는 것
js_script = "document.querySelector(\"#__next > div:nth-child(1) > div > div._2qKjD2._2Rfog3 > div > div._3ObMW8\").innerHTML"
raw = wd.execute_script("return " + js_script)
# print(type(raw))

# 내가 원하는 ul 부분의 dom 객체가 html 변수 안에 저장된다.
html = BeautifulSoup(raw, "html.parser")

# li.Q9_u2C 안에 원하는 정보가 있으므로 적절한 css Selector 문법을 활용해 필요한 정보를 얻는다.
# title : _2gfp-T
# addr : _2EFNlx
buttons = html.select("li.Q9_u2C span._2gfp-T")

def get_text(item):
    return item.text

title = list(map(get_text, buttons))

# html = wd.find_element(By.CSS_SELECTOR, "html")
# tag = html.find_element(By.CSS_SELECTOR, "li.Q9_u2C")
print(len(title))
print(title)